In [1]:
%config Completer.use_jedi = False
# %load_ext autoreload
# %autoreload 2

In [2]:
import os
import random
from tqdm import tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision

from src import utils
from src import pytorch_utils as ptu
from config import cfg

import warnings
warnings.filterwarnings("ignore")

In [3]:
cfg.tqdm_bar = True
cfg.prints = 'display'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device', device)

device cpu


In [25]:
dataset = torchvision.datasets.CIFAR10(root=cfg.data_path, train=False)

In [5]:
versions = [
    'no_rotation_resnet34_adam_lr0.0003_bs32',
    'rotation_resnet34_adam_lr0.0003_bs32',
]
epoch = -1

In [6]:
# log = pd.DataFrame(columns=['model', 'model_epoch', 'img', 'label', 'pred', 'loss', 'augment', 'angle'])
log = pd.read_csv('results_log.csv')

In [7]:
log

,model,model_epoch,img,label,pred,score,loss,augment,angle
0,rotation_resnet34_adam_lr0.0003_bs32,-1,0,3,3,1,1.237721,rotate,0
1,rotation_resnet34_adam_lr0.0003_bs32,-1,1,8,8,1,0.417182,rotate,0
2,rotation_resnet34_adam_lr0.0003_bs32,-1,2,8,8,1,0.278200,rotate,0
3,rotation_resnet34_adam_lr0.0003_bs32,-1,3,0,3,0,2.661825,rotate,0
4,rotation_resnet34_adam_lr0.0003_bs32,-1,4,6,6,1,0.718933,rotate,0
...,...,...,...,...,...,...,...,...,...
199995,no_rotation_resnet34_adam_lr0.0003_bs32,-1,9995,8,0,0,1.072774,rotate,90
199996,no_rotation_resnet34_adam_lr0.0003_bs32,-1,9996,3,5,0,2.098405,rotate,90
199997,no_rotation_resnet34_adam_lr0.0003_bs32,-1,9997,5,5,1,1.231478,rotate,90
199998,no_rotation_resnet34_adam_lr0.0003_bs32,-1,9998,1,5,0,2.610535,rotate,90


In [8]:
def eval_model(log, versions, epoch, angles, device, batch_size=1):
    if not batch_size == 1:
        raise NotImplementedError
    for version in versions:
        checkpoint = ptu.load_model(device, version=version, models_dir=cfg.models_dir, epoch=epoch)
        checkpoint.model.eval()
        checkpoint.model.to(device)
        with torch.no_grad():
            for angle in angles:
                transforms = torchvision.transforms.Compose([
                    utils.RotateAngle(angles=(angle, )),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize(utils.cifar10_mean, utils.cifar10_std),
                ])

                dataset = torchvision.datasets.CIFAR10(root=cfg.data_path, train=False, transform=transforms)

                loader = torch.utils.data.DataLoader(dataset,
                                                     batch_size=batch_size,
                                                     num_workers=cfg.num_workers,
                                                     shuffle=False,
                                                     drop_last=True)

                pbar = tqdm(loader)
                pbar.set_description(f'version={version}, angle={angle}')
                for i, batch in enumerate(pbar):
                    loss, results, _ = checkpoint.batch_pass(device, batch)
                    row = {
                        'model': checkpoint.version,
                        'model_epoch': epoch,
                        'img': i,
                        'label': results['trues'][0],
                        'pred': results['preds'][0],
                        'score': int(results['preds'][0] == results['trues'][0]),
                        'loss': float(loss.data),
                        'augment': 'rotate',
                        'angle': angle,
                    }
                    log = log.append(pd.Series(row), ignore_index=True)
                log.to_csv('results_log.csv', index=False)
    return log

In [9]:
angles = tuple(range(0, 100, 10))
# angles = (0, )

In [10]:
log = eval_model(log, versions, epoch, angles, device, batch_size=1)

version=no_rotation_resnet34_adam_lr0.0003_bs32, angle=0: 100%|██████████| 10000/10000 [02:29<00:00, 66.99it/s]
version=no_rotation_resnet34_adam_lr0.0003_bs32, angle=10: 100%|██████████| 10000/10000 [02:51<00:00, 58.42it/s]
version=no_rotation_resnet34_adam_lr0.0003_bs32, angle=20: 100%|██████████| 10000/10000 [03:06<00:00, 53.72it/s]
version=no_rotation_resnet34_adam_lr0.0003_bs32, angle=30: 100%|██████████| 10000/10000 [03:21<00:00, 49.52it/s]
version=no_rotation_resnet34_adam_lr0.0003_bs32, angle=40: 100%|██████████| 10000/10000 [03:43<00:00, 44.82it/s]
version=no_rotation_resnet34_adam_lr0.0003_bs32, angle=50: 100%|██████████| 10000/10000 [03:47<00:00, 43.95it/s]
version=no_rotation_resnet34_adam_lr0.0003_bs32, angle=60: 100%|██████████| 10000/10000 [03:52<00:00, 42.94it/s]
version=no_rotation_resnet34_adam_lr0.0003_bs32, angle=70: 100%|██████████| 10000/10000 [04:08<00:00, 40.30it/s]
version=no_rotation_resnet34_adam_lr0.0003_bs32, angle=80: 100%|██████████| 10000/10000 [04:26<00

In [ ]:
def f(string):
    if string == 'rotation_resnet34_adam_lr0.0003_bs32':
        return True
    elif string == 'no_rotation_resnet34_adam_lr0.0003_bs32':
        return False
    else:
        raise
log['rotations_train'] = log['model'].apply(f)
log['class'] = log['label'].apply(lambda x: dataset.classes[x])

In [33]:
log.pivot_table(values=['loss', 'score'], index='angle', columns='rotations_train')

loss             score        
rotations_train     False     True    False   True 
angle                                              
0                1.532003  1.887110  0.4840  0.3977
10               1.819066  1.711054  0.3889  0.4149
20               2.042534  1.687543  0.2923  0.4234
30               2.138657  1.693869  0.2665  0.4113
40               2.163460  1.695644  0.2612  0.4142
50               2.208765  1.703492  0.2467  0.4127
60               2.272597  1.717968  0.2278  0.4089
70               2.304710  1.714091  0.2212  0.4036
80               2.283663  1.722860  0.2579  0.4083
90               2.168132  1.818375  0.2849  0.3819

In [34]:
log.pivot_table(values=['loss', 'score'], index='class', columns='rotations_train')

loss             score        
rotations_train     False     True    False   True 
class                                              
airplane         2.123839  1.557869  0.3315  0.4765
automobile       2.051601  1.779763  0.2673  0.3446
bird             2.191143  1.973330  0.2275  0.3062
cat              2.047516  1.755063  0.2465  0.3891
deer             2.171195  1.956255  0.1941  0.3116
dog              2.318940  1.844591  0.2217  0.3344
frog             1.514352  1.609880  0.5785  0.5216
horse            2.441635  1.707918  0.2074  0.4291
ship             2.399218  1.577353  0.2113  0.5035
truck            1.674149  1.589984  0.4456  0.4603

In [39]:
for angle in angles:
    print(f'comparison on angle {angle} only')
    display(pd.pivot_table(log[log['angle'] == angle], values=['loss', 'score'], index='class', columns='rotations_train'))

comparison on angle 0 only


loss            score       
rotations_train     False     True   False  True 
class                                            
airplane         1.559209  1.856608  0.514  0.411
automobile       1.399124  2.005408  0.507  0.283
bird             1.590236  2.108033  0.441  0.300
cat              1.951770  1.774098  0.267  0.376
deer             1.614953  2.030758  0.457  0.333
dog              1.412443  1.802734  0.528  0.403
frog             1.435945  1.754875  0.562  0.509
horse            1.629953  1.670099  0.464  0.464
ship             1.336867  2.067672  0.569  0.456
truck            1.389529  1.800818  0.531  0.442

comparison on angle 10 only


loss            score       
rotations_train     False     True   False  True 
class                                            
airplane         2.005715  1.697559  0.390  0.407
automobile       1.570046  1.680444  0.410  0.351
bird             2.237139  2.091847  0.210  0.283
cat              2.073400  1.777841  0.245  0.385
deer             1.950460  1.939978  0.359  0.309
dog              1.613132  1.812500  0.431  0.351
frog             1.720813  1.704090  0.509  0.487
horse            1.790364  1.509837  0.395  0.494
ship             2.095101  1.588930  0.315  0.498
truck            1.134486  1.307510  0.625  0.584

comparison on angle 20 only


...

loss            score       
rotations_train     False     True   False  True 
class                                            
airplane         2.508587  1.474447  0.253  0.479
automobile       2.360215  1.729162  0.168  0.342
bird             2.500081  2.053359  0.109  0.249
cat              1.960247  1.760698  0.238  0.390
deer             2.503818  1.877081  0.064  0.348
dog              2.534287  1.963668  0.099  0.266
frog             1.310046  1.516147  0.655  0.549
horse            2.682368  1.732774  0.137  0.431
ship             2.723774  1.450033  0.125  0.518
truck            1.963681  1.583539  0.364  0.464

comparison on angle 80 only


loss            score       
rotations_train     False     True   False  True 
class                                            
airplane         2.363115  1.584517  0.352  0.467
automobile       2.607536  1.767318  0.142  0.319
bird             2.300267  2.080532  0.219  0.271
cat              1.962306  1.761210  0.250  0.389
deer             2.176079  1.852744  0.237  0.340
dog              2.031201  1.915097  0.275  0.271
frog             1.722365  1.672774  0.494  0.500
horse            2.701211  1.562264  0.124  0.497
ship             3.031121  1.577880  0.083  0.494
truck            1.941429  1.454261  0.403  0.535

comparison on angle 90 only


loss            score       
rotations_train     False     True   False  True 
class                                            
airplane         2.099855  1.655473  0.330  0.454
automobile       2.544794  2.058349  0.191  0.256
bird             1.667915  2.033160  0.398  0.290
cat              2.003581  1.806367  0.255  0.356
deer             1.968093  1.976246  0.260  0.303
dog              1.761334  1.790437  0.409  0.368
frog             1.625270  1.663468  0.498  0.516
horse            2.954783  1.758120  0.071  0.413
ship             2.651989  1.668699  0.178  0.481
truck            2.403705  1.773433  0.259  0.382